In [146]:
import pydicom as dicom
import os
import shutil
import numpy as np

In [121]:
project_dir = "/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split"
output_dir = "/media/nraresearch/Elements/Dual_Split_Projekt_ETH/preprocessed_dual_split"

list_studies = [ name for name in os.listdir(project_dir) if os.path.isdir(os.path.join(project_dir, name))  ]

In [122]:
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    

In [126]:
# functions 

def copy_dcm_file(source, destination):
    """
    Source path
    Destination path: destination
    """
    try:
        shutil.copyfile(source, destination)
        #print("File copied successfully.")

    # If source and destination are same
    except shutil.SameFileError:
        print("Source and destination represents the same file.")

    # If destination is a directory.
    except IsADirectoryError:
        print("Destination is a directory: %s" %destination)

    # If there is any permission issue
    except PermissionError:
        print("Permission denied.")

    # For other errors
    except:
        print("Error occurred while copying file.")
    return

In [216]:
path_out_patient

'/media/nraresearch/Elements/Dual_Split_Projekt_ETH/preprocessed_dual_split/100'

In [300]:
for i_dir in list_studies:
    #i_dir = "10"

    path_out_patient = os.path.join(output_dir, i_dir)
    path_in_patient = os.path.join(project_dir, i_dir)

    if os.path.isdir(path_out_patient):
        print("patient %s exist"%i_dir)
        continue
    else:
        os.mkdir(path_out_patient)
        
    
    print("processing study: %s" % i_dir)
    if not os.path.isdir(os.path.join(path_out_patient,"33")): os.mkdir(os.path.join(path_out_patient,"33"))
    if not os.path.isdir(os.path.join(path_out_patient,"66")): os.mkdir(os.path.join(path_out_patient,"66"))
    if not os.path.isdir(os.path.join(path_out_patient,"100")): os.mkdir(os.path.join(path_out_patient,"100"))

    print(path_in_patient)
    counter = 0
    c_failure = 0

    max_slice_33 = 0
    max_series33 = []
    max_slice_66 = 0
    max_series66 = []
    max_slice_100 = 0
    max_series100 = []


    for root, dir, files in os.walk(path_in_patient):
        for i_file in files:
            path_i_file = os.path.join(path_in_patient, i_file)
            ds = dicom.read_file(path_i_file)
            try:
                if ds.ImageType[6] == "DET_A" and ds.ImageType[0] == "ORIGINAL":
                    path_det_a = os.path.join(output_dir, i_dir, "66", str(ds.SeriesNumber), i_file)
                    if ds.InstanceNumber >= max_slice_66: max_slice_66 = ds.InstanceNumber

                elif ds.ImageType[6] == "DET_B" and ds.ImageType[0] == "ORIGINAL":
                    path_det_b = os.path.join(output_dir, i_dir, "33", str(ds.SeriesNumber), i_file)
                    if ds.InstanceNumber >= max_slice_33: max_slice_33 = ds.InstanceNumber 

                elif ds.ImageType[6] == "DET_AB" and ds.ImageType[0] == "ORIGINAL":
                    path_det_ab = os.path.join(output_dir, i_dir, "100",str(ds.SeriesNumber), i_file)
                    if ds.InstanceNumber >= max_slice_100: max_slice_100 = ds.InstanceNumber 
            except:
                pass
    
    print(max_slice_33)
    print(max_slice_66)
    print(max_series100)
    
    if len({max_slice_33, max_slice_66, max_slice_100}) != 1:
        min_slice_threshold = min(max_slice_33, max_slice_66, max_slice_100)
        if max_slice_33 != min_slice_threshold:       
            max_slice_33 = min_slice_threshold
            print("WARNING! The filter of the sequence at patient %s, in dose 33 has been lowered"%i_dir)
        if max_slice_66 != min_slice_threshold:       
            max_slice_66 = min_slice_threshold
            print("WARNING! The filter of the sequence at patient %s, in dose 66 has been lowered"%i_dir)
        if max_slice_100 != min_slice_threshold:       
            max_slice_100 = min_slice_threshold
            print("WARNING! The filter of the sequence at patient %s, in dose 100 has been lowered"%i_dir)
                
    print("Sequences before")
    print(max_series33)
    print(max_series66)
    print(max_series100)
            
    # find the sequences in each subfolder that has the biggest number of slices
    for root, dir, files in os.walk(path_in_patient):
        for i_file in files:
            path_i_file = os.path.join(path_in_patient, i_file)
            ds = dicom.read_file(path_i_file)
            
            if len({max_slice_33, max_slice_66, max_slice_100}) == 1:
                try:
                    if ds.ImageType[6] == "DET_A" and ds.ImageType[0] == "ORIGINAL":
                        path_det_a = os.path.join(output_dir, i_dir, "66", str(ds.SeriesNumber), i_file)
                        if ds.InstanceNumber == max_slice_66: 
                            if ds.SeriesNumber not in max_series66: max_series66+= [ds.SeriesNumber ] 
                        if ds.InstanceNumber > min_slice_threshold: 
                            if ds.SeriesNumber in max_series66: max_series66.remove(ds.SeriesNumber)  

                    elif ds.ImageType[6] == "DET_B" and ds.ImageType[0] == "ORIGINAL":
                        path_det_b = os.path.join(output_dir, i_dir, "33", str(ds.SeriesNumber), i_file)
                        if ds.InstanceNumber == max_slice_33: 
                            if ds.SeriesNumber not in max_series33: max_series33+= [ds.SeriesNumber ] 
                        if ds.InstanceNumber > min_slice_threshold: 
                            if ds.SeriesNumber in max_series33: max_series33.remove(ds.SeriesNumber)    

                    elif ds.ImageType[6] == "DET_AB" and ds.ImageType[0] == "ORIGINAL":
                        path_det_ab = os.path.join(output_dir, i_dir, "100",str(ds.SeriesNumber), i_file)
                        if ds.InstanceNumber == max_slice_100: 
                            if ds.SeriesNumber not in max_series100: max_series100+= [ds.SeriesNumber ]
                        if ds.InstanceNumber > min_slice_threshold: 
                            if ds.SeriesNumber in max_series100: max_series100.remove(ds.SeriesNumber)
                except:
                        pass
            else:
                print("WARNING! Sequence with max slice are not present in all the subfolder at patient %s!" % i_dir)

                
    print("Sequences after")
    print(max_series33)
    print(max_series66)
    print(max_series100)


    if len({max_slice_33, max_slice_66, max_slice_100}) == 1:
        # delete the series that have not the same number of slices as max and not the same as in other subfolders (33,66,100)
        for root, dir, files in os.walk(path_in_patient):
            for i_file in files:
                path_i_file = os.path.join(path_in_patient, i_file)
                ds = dicom.read_file(path_i_file)
                #print(os.path.join(path_in_patient, i_file))
                try:
                    if ds.ImageType[6] == "DET_A" and ds.ImageType[0] == "ORIGINAL" and ds.SeriesNumber in max_series66 :
                        path_det_a = os.path.join(output_dir, i_dir, "66", str(ds.SeriesNumber), i_file)
                        seq_dir = os.path.join(output_dir, i_dir, "66", str(ds.SeriesNumber))
                        if not os.path.isdir(seq_dir): os.mkdir(seq_dir)
                        if not os.path.isfile(path_det_a): copy_dcm_file(source = path_i_file, destination = path_det_a)

                    elif ds.ImageType[6] == "DET_B" and ds.ImageType[0] == "ORIGINAL" and ds.SeriesNumber in max_series33:
                        path_det_b = os.path.join(output_dir, i_dir, "33", str(ds.SeriesNumber), i_file)
                        seq_dir = os.path.join(output_dir, i_dir, "33", str(ds.SeriesNumber))
                        if not os.path.isdir(seq_dir): os.mkdir(seq_dir)
                        if not os.path.isfile(path_det_b): copy_dcm_file(source = path_i_file, destination = path_det_b)

                    elif ds.ImageType[6] == "DET_AB" and ds.ImageType[0] == "ORIGINAL" and ds.SeriesNumber in max_series100:
                        seq_dir = os.path.join(output_dir, i_dir, "100", str(ds.SeriesNumber))
                        if not os.path.isdir(seq_dir): os.mkdir(seq_dir)
                        path_det_ab = os.path.join(output_dir, i_dir, "100",str(ds.SeriesNumber), i_file)
                        if not os.path.isfile(path_det_ab): copy_dcm_file(source = path_i_file, destination = path_det_ab)
                    else:
                        counter +=1
                except:
                    c_failure += 1
                    print("Failure B")
                    #print("'FileDataset' %s could not be processed.  has it no attribute 'ImageType'? " % i_file)
    else:
        for root, dir, files in os.walk(path_in_patient):
            for i_file in files:
                path_i_file = os.path.join(path_in_patient, i_file)
                ds = dicom.read_file(path_i_file)
                #print(os.path.join(path_in_patient, i_file))
                try:
                    if ds.ImageType[6] == "DET_A" and ds.ImageType[0] == "ORIGINAL":
                        path_det_a = os.path.join(output_dir, i_dir, "66", str(ds.SeriesNumber), i_file)
                        seq_dir = os.path.join(output_dir, i_dir, "66", str(ds.SeriesNumber))
                        if not os.path.isdir(seq_dir): os.mkdir(seq_dir)
                        if not os.path.isfile(path_det_a): copy_dcm_file(source = path_i_file, destination = path_det_a)

                    elif ds.ImageType[6] == "DET_B" and ds.ImageType[0] == "ORIGINAL":
                        path_det_b = os.path.join(output_dir, i_dir, "33", str(ds.SeriesNumber), i_file)
                        seq_dir = os.path.join(output_dir, i_dir, "33", str(ds.SeriesNumber))
                        if not os.path.isdir(seq_dir): os.mkdir(seq_dir)                    
                        if not os.path.isfile(path_det_b): copy_dcm_file(source = path_i_file, destination = path_det_b)

                    elif ds.ImageType[6] == "DET_AB" and ds.ImageType[0] == "ORIGINAL":
                        path_det_ab = os.path.join(output_dir, i_dir, "100",str(ds.SeriesNumber), i_file)
                        seq_dir = os.path.join(output_dir, i_dir, "100", str(ds.SeriesNumber))
                        if not os.path.isdir(seq_dir): os.mkdir(seq_dir)
                        if not os.path.isfile(path_det_ab): copy_dcm_file(source = path_i_file, destination = path_det_ab)
                    else:
                        counter +=1
                except:
                    c_failure += 1
                    #print("'FileDataset' %s could not be processed.  has it no attribute 'ImageType'? " % i_file)


    print("number of files that has not bee copied: %i" % counter)
    print("number of failures occured during processing.E.g. file does not have metaheader: %i" % c_failure)


patient 1 exist
patient 10 exist
patient 100 exist
patient 101 exist
patient 102 exist
patient 103 exist
patient 104 exist
patient 105 exist
patient 106 exist
patient 107 exist
patient 108 exist
processing study: 109
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/109
336
336
[]
WARNING! The filter of the sequence at patient 109, in dose 100 has been lowered
Sequences before
[]
[]
[]
Sequences after
["7"]
["6"]
["9"]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 717
number of failures occured during processing.E.g. file does not have metaheader: 9
patient 11 exist
processing study: 110
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/110
321
321
[]
Sequences before
[]
[]
[]
Sequences after
["7"]
["6"]
["5", "8"]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 167
number of failures occured 

319
319
[]
Sequences before
[]
[]
[]
Sequences after
[]
[]
[]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 1445
number of failures occured during processing.E.g. file does not have metaheader: 6
processing study: 41
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/41
289
289
[]
WARNING! The filter of the sequence at patient 41, in dose 100 has been lowered
Sequences before
[]
[]
[]
Sequences after
["7"]
["6"]
["8"]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 456
number of failures occured during processing.E.g. file does not have metaheader: 6
processing study: 42
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/42
329
329
[]
Sequences before
[]
[]
[]
Sequences after
[]
[]
[]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 1477
number of failures occured during pr

number of files that has not bee copied: 711
number of failures occured during processing.E.g. file does not have metaheader: 7
processing study: 61
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/61
323
323
[]
Sequences before
[]
[]
[]
Sequences after
["7"]
["6"]
["5", "8"]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 60
number of failures occured during processing.E.g. file does not have metaheader: 6
processing study: 62
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/62
337
337
[]
WARNING! The filter of the sequence at patient 62, in dose 100 has been lowered
Sequences before
[]
[]
[]
Sequences after
["7"]
["6"]
["8"]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 525
number of failures occured during processing.E.g. file does not have metaheader: 6
processing study: 63
/media/nraresearch/Elements/Dual_Split_Proj

361
361
[]
Sequences before
[]
[]
[]
Sequences after
[]
[]
[]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 1836
number of failures occured during processing.E.g. file does not have metaheader: 7
processing study: 81
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/81
249
249
[]
WARNING! The filter of the sequence at patient 81, in dose 100 has been lowered
Sequences before
[]
[]
[]
Sequences after
["7"]
["6"]
["8"]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 415
number of failures occured during processing.E.g. file does not have metaheader: 6
processing study: 82
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/82
290
290
[]
Sequences before
[]
[]
[]
Sequences after
[]
[]
[]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 1308
number of failures occured

352
352
[]
Sequences before
[]
[]
[]
Sequences after
[]
[]
[]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 1715
number of failures occured during processing.E.g. file does not have metaheader: 7
processing study: 120
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/120
281
281
[]
WARNING! The filter of the sequence at patient 120, in dose 100 has been lowered
Sequences before
[]
[]
[]
Sequences after
["7"]
["6"]
["8"]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 497
number of failures occured during processing.E.g. file does not have metaheader: 6
processing study: 121
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/121
284
284
[]
Sequences before
[]
[]
[]
Sequences after
[]
[]
[]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 1465
number of f

...

293
293
[]
Sequences before
[]
[]
[]
Sequences after
["7"]
["6"]
["5", "8"]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 498
number of failures occured during processing.E.g. file does not have metaheader: 7
processing study: 200
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/200
346
346
[]
Sequences before
[]
[]
[]
Sequences after
[]
[]
[]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 1686
number of failures occured during processing.E.g. file does not have metaheader: 8
processing study: 202
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/202
300
300
[]
Sequences before
[]
[]
[]
Sequences after
["8"]
["7"]
["9", "13"]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 610
number of failures occured during processing.E.g. fil

Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B


305
305
[]
Sequences before
[]
[]
[]
Sequences after
[]
[]
[]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 1531
number of failures occured during processing.E.g. file does not have metaheader: 8
processing study: 211
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/211
352
352
[]
Sequences before
[]
[]
[]
Sequences after
[]
[]
[]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 1747
number of failures occured during processing.E.g. file does not have metaheader: 8
processing study: 212
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/212
318
318
[]
WARNING! The filter of the sequence at patient 212, in dose 100 has been lowered
Sequences before
[]
[]
[]
Sequences after
["7"]
["6"]
["8"]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee cop

Sequences after
["7"]
["6"]
["8"]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 525
number of failures occured during processing.E.g. file does not have metaheader: 8
processing study: 23
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/23
380
380
[]
Sequences before
[]
[]
[]
Sequences after
[]
[]
[]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
number of files that has not bee copied: 1798
number of failures occured during processing.E.g. file does not have metaheader: 8
processing study: 230
/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/230
326
326
[]
Sequences before
[]
[]
[]
Sequences after
[]
[]
[]
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure

Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B
Failure B


In [297]:
ds

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 214
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.1.4.75432.30000020061807274821200005896
(0002, 0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.30071.8
(0002, 0013) Implementation Version Name         SH: 'SmartAnonymize'
(0002, 0016) Source Application Entity Title     AE: 'EE2001664RADQR'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI DUAL', 'STD', 'DNRG', 'DET_A']
(0008, 0016) SOP Class UID                      

In [299]:
ds.ImageType[6] == "DET_A" and ds.ImageType[0] == "ORIGINAL" and ds.SeriesNumber in max_series66

True

In [296]:
if ds.ImageType[6] == "DET_A" and ds.ImageType[0] == "ORIGINAL" and ds.SeriesNumber in max_series33 :
    path_det_a = os.path.join(output_dir, i_dir, "66", str(ds.SeriesNumber), i_file)
    seq_dir = os.path.join(output_dir, i_dir, "66", str(ds.SeriesNumber))
    if not os.path.isdir(seq_dir): os.mkdir(seq_dir)
    if not os.path.isfile(path_det_a): copy_dcm_file(source = path_i_file, destination = path_det_a)

elif ds.ImageType[6] == "DET_B" and ds.ImageType[0] == "ORIGINAL" and ds.SeriesNumber in max_series66:
    path_det_b = os.path.join(output_dir, i_dir, "33", str(ds.SeriesNumber), i_file)
    seq_dir = os.path.join(output_dir, i_dir, "33", str(ds.SeriesNumber))
    if not os.path.isdir(seq_dir): os.mkdir(seq_dir)
    if not os.path.isfile(path_det_b): copy_dcm_file(source = path_i_file, destination = path_det_b)

elif ds.ImageType[6] == "DET_AB" and ds.ImageType[0] == "ORIGINAL" and ds.SeriesNumber in max_series100:
    seq_dir = os.path.join(output_dir, i_dir, "100", str(ds.SeriesNumber))
    if not os.path.isdir(seq_dir): os.mkdir(seq_dir)
    path_det_ab = os.path.join(output_dir, i_dir, "100",str(ds.SeriesNumber), i_file)
    if not os.path.isfile(path_det_ab): copy_dcm_file(source = path_i_file, destination = path_det_ab)
else:
    counter +=1

In [291]:
len({max_slice_33, max_slice_66, max_slice_100}) == 1

True

In [271]:
ds.SeriesNumber

"7"

In [278]:
max_series100

["8"]

In [273]:
max_series100 += [ds.SeriesNumber]

In [277]:
max_series100.remove(ds.SeriesNumber)

In [250]:
if len({max_slice_33, max_slice_66, max_slice_100}) is False:
    min_slice_threshold = min(max_slice_33, max_slice_66, max_slice_100)
    if max_slice_33 != min_slice_threshold:       
        max_slice_33 = min_slice_threshold
        print("WARNING! The filter of the sequence at patient %i_dir, in dose 33 has been lowered")
    if max_slice_66 != min_slice_threshold:       
        max_slice_66 = min_slice_threshold
        print("WARNING! The filter of the sequence at patient %i_dir, in dose 66 has been lowered")
    if max_slice_100 != min_slice_threshold:       
        max_slice_100 = min_slice_threshold
        print("WARNING! The filter of the sequence at patient %i_dir, in dose 100 has been lowered")

In [256]:

len({max_slice_33, max_slice_66, max_slice_100})==1:
    
    print("k")

In [ ]:
max_slice_33, max_slice_66, max_slice_100
len({max_slice_33, max_slice_66, max_slice_100}) 

In [257]:
len({max_slice_33, max_slice_66, max_slice_100})

2

In [251]:
if max_slice_100 != min_slice_threshold:       
    max_slice_100 = min_slice_threshold


In [252]:
max_slice_66

"287"

In [255]:
max_slice_100 =326

In [253]:
max_slice_100

"287"

In [220]:
print(max_slice_33, max_slice_66, max_slice_100)

287 287 302


In [203]:
ds.ImageType[6] == "DET_AB" and ds.ImageType[0] == "ORIGINAL" and ds.SeriesNumber in max_series100
path_det_ab

'/media/nraresearch/Elements/Dual_Split_Projekt_ETH/preprocessed_dual_split/10/100/15/1.3.12.2.1107.5.1.4.75432.30000020041606585187500017536.dcm'

In [153]:
ds.ImageType[6] == "DET_AB" and ds.ImageType[0] == "ORIGINAL" 

True

In [198]:
max_series100 = []
max_series100 += [ds.SeriesNumber]
if ds.SeriesNumber not in max_series100: max_series100 += [ds.SeriesNumber]
max_series100


["15"]

In [184]:
k = []
k+= ds.SeriesNumber
k

TypeError: 'IS' object is not iterable

In [173]:
k=list()

In [175]:
k = k.append(str(ds.SeriesNumber))
print(k)

AttributeError: 'NoneType' object has no attribute 'append'

In [142]:
a, b, c = 3, 3, 3
max_slice_33

True

In [87]:
ds = dicom.read_file(os.path.join(path_in_patient, i_file))
ds.ImageType[6] == "DET_AB"

True

In [89]:
path_det_ab = os.path.join(output_dir, i_dir, "100")
if not os.path.isdir(path_det_ab): os.mkdir(path_det_ab)
copy_dcm_file(source = path_i_file, destination = path_det_ab)

Error occurred while copying file.


In [90]:
path_i_file

'/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/1/1.3.12.2.1107.5.1.4.75432.30000020061507080076200016909.dcm'

In [91]:
path_det_ab

'/media/nraresearch/Elements/Dual_Split_Projekt_ETH/preprocessed_dual_split/1/100'

In [80]:
path_det_b

'/media/nraresearch/Elements/Dual_Split_Projekt_ETH/preprocessed_dual_split/1/33'

In [99]:
filename = "1.3.12.2.1107.5.1.4.75432.30000020061507080076200016414.dcm"
path_file = os.path.join("/media/nraresearch/Elements/Dual_Split_Projekt_ETH/Anonmyized_final_dual_split/1", filename)

ds = dicom.read_file(path_file)


In [100]:
ds

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 214
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.1.4.75432.30000020061507080076200016414
(0002, 0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.30071.8
(0002, 0013) Implementation Version Name         SH: 'SmartAnonymize'
(0002, 0016) Source Application Entity Title     AE: 'EE2001664RADQR'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI DUAL', 'STD', 'DNRG', 'DET_AB']
(0008, 0016) SOP Class UID                     

In [101]:
ds.ImageType[6]

'DET_AB'

In [53]:
filename = "1.3.12.2.1107.5.1.4.75432.30000020061507080076200015958.dcm"
path_file = os.path.join("/media/nraresearch/Elements/Dual_Split_Test_For_Bence/33%/PP Lunge Inspi 1.5  Bl64  3 LCAD  B_100kV", filename)

ds = dicom.read_file(path_file)
# 33%


Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 212
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.1.4.75432.30000020061507080076200015958
(0002, 0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.30071.8
(0002, 0013) Implementation Version Name         SH: 'fo-dicom 4.0.4'
(0002, 0016) Source Application Entity Title     AE: 'SYNEDRA_AIM'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI DUAL', 'STD', 'DNRG', 'DET_B']
(0008, 0016) SOP Class UID                       UI

In [57]:
ds.ImageType[6]

'DET_B'

In [58]:
filename = "1.3.12.2.1107.5.1.4.75432.30000020061507080076200015587.dcm"
path_file = os.path.join("/media/nraresearch/Elements/Dual_Split_Test_For_Bence/66%/PP Lunge Inspi 1.5  Bl64  3 LCAD  A_100kV", filename)

ds = dicom.read_file(path_file)
# 66% det_A
ds

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 212
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.1.4.75432.30000020061507080076200015587
(0002, 0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.30071.8
(0002, 0013) Implementation Version Name         SH: 'fo-dicom 4.0.4'
(0002, 0016) Source Application Entity Title     AE: 'SYNEDRA_AIM'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI DUAL', 'STD', 'DNRG', 'DET_A']
(0008, 0016) SOP Class UID                       UI

In [59]:
ds.ImageType[6]

'DET_A'

In [131]:
# d 100 series 15 - should keep
filename = "1.3.12.2.1107.5.1.4.75432.30000020041606585187500017377.dcm"
path_file = os.path.join("/media/nraresearch/Elements/Dual_Split_Projekt_ETH/preprocessed_dual_split/10/100", filename)
ds = dicom.read_file(path_file)
ds

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 214
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.1.4.75432.30000020041606585187500017377
(0002, 0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.30071.8
(0002, 0013) Implementation Version Name         SH: 'SmartAnonymize'
(0002, 0016) Source Application Entity Title     AE: 'EE2001664RADQR'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI DUAL', 'STD', 'DNRG', 'DET_AB']
(0008, 0016) SOP Class UID                     

In [132]:
# patient 10, d 100 series 7- should be dropped out
filename = "1.3.12.2.1107.5.1.4.75432.30000020041606585187500010074.dcm"
path_file = os.path.join("/media/nraresearch/Elements/Dual_Split_Projekt_ETH/preprocessed_dual_split/10/100", filename)
ds = dicom.read_file(path_file)
ds

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 214
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.1.4.75432.30000020041606585187500010074
(0002, 0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.30071.8
(0002, 0013) Implementation Version Name         SH: 'SmartAnonymize'
(0002, 0016) Source Application Entity Title     AE: 'EE2001664RADQR'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI DUAL', 'STD', 'DNRG', 'DET_AB']
(0008, 0016) SOP Class UID                     

In [133]:
# patient 10,  d100 series 7- should be dropped out
filename = "1.3.12.2.1107.5.1.4.75432.30000020041606585187500009439.dcm"
path_file = os.path.join("/media/nraresearch/Elements/Dual_Split_Projekt_ETH/preprocessed_dual_split/10/100", filename)
ds = dicom.read_file(path_file)
ds

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 214
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.1.4.75432.30000020041606585187500009439
(0002, 0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.30071.8
(0002, 0013) Implementation Version Name         SH: 'SmartAnonymize'
(0002, 0016) Source Application Entity Title     AE: 'EE2001664RADQR'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI DUAL', 'STD', 'SNRG', 'DET_AB']
(0008, 0016) SOP Class UID                     